In [6]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import textwrap  #
import joblib
import math 
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score, top_k_accuracy_score
)
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from sklearn.calibration import calibration_curve

from sklearn.model_selection import train_test_split
from mambular.models import MambularClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score, top_k_accuracy_score
)
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from sklearn.calibration import calibration_curve
np.random.seed(0)

In [12]:
data = pd.read_csv('train_pre_txbf.csv')

In [4]:
# Number of samples
n_samples = 20369
n_features = 22

In [15]:

selected_cols = ["mcs","raw_h_sv2_mean", "raw_h_mag_mean", "raw_csi_matrix_r0_c0_mag_p95", "raw_csi_matrix_r0_c3_mag_p95", "raw_csi_matrix_r1_c3_mag_std", "raw_csi_matrix_r1_c1_spec_flatness", "raw_sinr_txbf_sum_mean", "raw_h_sv1_mean", "raw_csi_matrix_r1_c1_spec_entropy", "raw_csi_matrix_r0_c0_spec_entropy", "raw_h_sv2_std", "raw_csi_matrix_r0_c3_mag_p50", "raw_pdp_rms", "raw_csi_matrix_r0_c2_mag_max", "raw_csi_matrix_r1_c0_mag_p50", "raw_csi_matrix_r0_c3_spec_energy", "raw_noise_floor", "raw_csi_matrix_r1_c1_mag_min", "raw_csi_matrix_r0_c2_mag_p05", "raw_csi_matrix_r1_c3_mag_p95", "raw_h_mag_std", "raw_csi_matrix_r0_c1_mag_min", "raw_csi_matrix_r0_c0_spec_flatness", "raw_csi_matrix_r1_c0_spec_flatness", "raw_csi_matrix_r1_c2_mag_std", "raw_csi_matrix_r0_c1_mag_p50", "raw_csi_matrix_r1_c2_spec_flatness", "raw_csi_matrix_r1_c0_spec_entropy", "raw_csi_matrix_r0_c3_mag_max"]
    

data = data[selected_cols]


In [16]:
data.columns

Index(['mcs', 'raw_h_sv2_mean', 'raw_h_mag_mean',
       'raw_csi_matrix_r0_c0_mag_p95', 'raw_csi_matrix_r0_c3_mag_p95',
       'raw_csi_matrix_r1_c3_mag_std', 'raw_csi_matrix_r1_c1_spec_flatness',
       'raw_sinr_txbf_sum_mean', 'raw_h_sv1_mean',
       'raw_csi_matrix_r1_c1_spec_entropy',
       'raw_csi_matrix_r0_c0_spec_entropy', 'raw_h_sv2_std',
       'raw_csi_matrix_r0_c3_mag_p50', 'raw_pdp_rms',
       'raw_csi_matrix_r0_c2_mag_max', 'raw_csi_matrix_r1_c0_mag_p50',
       'raw_csi_matrix_r0_c3_spec_energy', 'raw_noise_floor',
       'raw_csi_matrix_r1_c1_mag_min', 'raw_csi_matrix_r0_c2_mag_p05',
       'raw_csi_matrix_r1_c3_mag_p95', 'raw_h_mag_std',
       'raw_csi_matrix_r0_c1_mag_min', 'raw_csi_matrix_r0_c0_spec_flatness',
       'raw_csi_matrix_r1_c0_spec_flatness', 'raw_csi_matrix_r1_c2_mag_std',
       'raw_csi_matrix_r0_c1_mag_p50', 'raw_csi_matrix_r1_c2_spec_flatness',
       'raw_csi_matrix_r1_c0_spec_entropy', 'raw_csi_matrix_r0_c3_mag_max'],
      dtype='object')

In [17]:
X = data.drop(columns=["mcs"])
y = data['mcs'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)

print("类映射：", dict(zip(le.classes_, range(len(le.classes_)))))
print("训练集标签范围:", y_train_enc.min(), "→", y_train_enc.max())
print("类别数:", len(le.classes_))

类映射： {137.6: 0, 206.5: 1, 275.3: 2, 309.7: 3, 344.1: 4, 412.9: 5, 458.8: 6, 516.2: 7, 573.5: 8, 619.4: 9, 688.2: 10}
训练集标签范围: 0 → 10
类别数: 11


In [20]:
import re
import pandas as pd

def sanitize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    new_cols = [str(c).replace("__", "_") for c in df.columns]
    new_cols = [re.sub(r"[^0-9a-zA-Z_]", "_", c) for c in new_cols]

    seen = {}
    final_cols = []
    for c in new_cols:
        if c not in seen:
            seen[c] = 0
            final_cols.append(c)
        else:
            seen[c] += 1
            final_cols.append(f"{c}_{seen[c]}")
    df.columns = final_cols
    return df

X_train = sanitize_columns(X_train)
X_test  = sanitize_columns(X_test)



In [22]:

class _TinyDWConvBlock(nn.Module):
    def __init__(self, d_model: int, kernel_size: int = 3, expansion: int = 2, dropout: float = 0.1):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.dw = nn.Conv1d(d_model, d_model, kernel_size=kernel_size, padding=kernel_size // 2, groups=d_model)
        self.pw1 = nn.Linear(d_model, expansion * d_model)
        self.act = nn.SiLU()
        self.pw2 = nn.Linear(expansion * d_model, d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):                 # x: (B, L, D)
        residual = x
        x = self.norm(x)
        x = x.transpose(1, 2)             # -> (B, D, L)
        x = self.dw(x)
        x = x.transpose(1, 2)             # -> (B, L, D)
        x = self.pw2(self.act(self.pw1(x)))
        x = self.drop(x)
        return residual + x


class _TinyDWConvMixer(nn.Module):
    def __init__(self, d_model: int, n_layers: int = 1, kernel_size: int = 3, expansion: int = 2, dropout: float = 0.1):
        super().__init__()
        self.blocks = nn.ModuleList(
            [_TinyDWConvBlock(d_model, kernel_size, expansion, dropout) for _ in range(max(1, n_layers))]
        )

    def forward(self, x):
        for blk in self.blocks:
            x = blk(x)
        return x


class _LearnedAttentionPool(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.q = nn.Parameter(torch.randn(d_model))
        self.proj = nn.Linear(d_model, d_model, bias=False)

    def forward(self, x):                 # x: (B, L, D)
        q = self.q[None, None, :]                                  # (1,1,D)
        scores = torch.einsum('bld,b1d->bl1', self.proj(x), q) / (x.size(-1) ** 0.5)
        attn = torch.softmax(scores, dim=1)                        # (B,L,1)
        return (x * attn).sum(dim=1)                               # (B,D)


class Mambular(BaseModel):


    _DEFAULTS = {
        "mixer_type": "dwconv",        # 'dwconv' | 'identity' | 'mamba'
        "n_mixer_layers": 1,
        "mixer_kernel_size": 3,
        "mixer_expansion": 2,
        "dropout": 0.1,
        "pooling_method": "mean",      # 'mean' | 'attn' | 'mean+attn'
        "shuffle_embeddings": False,
        "d_model_fallback": 128,     
        "mamba_version": "mamba-torch"
    }

    def __init__(
        self,
        feature_information: tuple,       # (cat_feature_info, num_feature_info, embedding_feature_info)
        num_classes: int = 1,
        config: DefaultMambularConfig = None,  
        **kwargs,
    ):

        if config is None:
            config = DefaultMambularConfig()
        if not hasattr(config, "d_model") or config.d_model is None:
            config.d_model = self._DEFAULTS["d_model_fallback"]


        for k, v in self._DEFAULTS.items():
            if not hasattr(config, k):
                setattr(config, k, v)

        super().__init__(config=config, **kwargs)
        self.save_hyperparameters(ignore=["feature_information"])
        self.returns_ensemble = False


        self.embedding_layer = EmbeddingLayer(*feature_information, config=config)

    
        mixer_type = self.hparams.mixer_type
        if mixer_type == "mamba":
            if self.hparams.mamba_version == "mamba-torch":
                self.mixer = Mamba(config)
            else:
                self.mixer = MambaOriginal(config)
        elif mixer_type == "identity":
            self.mixer = nn.Identity()
        else:
            self.mixer = _TinyDWConvMixer(
                d_model=self.hparams.d_model,
                n_layers=int(self.hparams.n_mixer_layers),
                kernel_size=int(self.hparams.mixer_kernel_size),
                expansion=int(self.hparams.mixer_expansion),
                dropout=float(self.hparams.dropout),
            )


        self.shuffle_embeddings = bool(self.hparams.shuffle_embeddings)
        if self.shuffle_embeddings:
            seq_len = int(getattr(self.embedding_layer, "seq_len", 0) or 0)
            if seq_len > 0:
                self.register_buffer("perm", torch.randperm(seq_len))
            else:
                self.perm = None


        self.pooling_method = self.hparams.pooling_method
        if "attn" in self.pooling_method:
            self.attn_pool = _LearnedAttentionPool(self.hparams.d_model)


        self.tabular_head = MLPhead(
            input_dim=self.hparams.d_model,
            config=config,
            output_dim=num_classes,
        )


        n_inputs = int(np.sum([len(info) for info in feature_information]))
        self.initialize_pooling_layers(config=config, n_inputs=n_inputs)

    def pool_sequence(self, x):
        if self.pooling_method == "mean":
            return x.mean(dim=1)
        elif self.pooling_method == "attn":
            return self.attn_pool(x)
        elif self.pooling_method == "mean+attn":
            return 0.5 * (x.mean(dim=1) + self.attn_pool(x))
        else:
            # 若父类定义了其它策略，则回退；否则用 mean
            try:
                return super().pool_sequence(x)
            except AttributeError:
                return x.mean(dim=1)

    def forward(self, *data):

        x = self.embedding_layer(*data)          # (B, L, D)

        if self.shuffle_embeddings and getattr(self, "perm", None) is not None:
            if self.perm.numel() == x.size(1):
                x = x[:, self.perm, :]

        x = self.mixer(x)                        # (B, L, D)
        x = self.pool_sequence(x)                # (B, D)
        preds = self.tabular_head(x)             # (B, C)
        return preds

In [23]:

from mambular.base_models.mambular import Mambular
from mambular.configs.mambular_config import DefaultMambularConfig
from mambular.utils.docstring_generator import generate_docstring
from mambular.models.utils.sklearn_base_classifier import SklearnBaseClassifier

class MambularClassifier(SklearnBaseClassifier):
    __doc__ = generate_docstring(
        DefaultMambularConfig,
        model_description="""
        Mambular classifier. This class extends the SklearnBaseClassifier class and uses the Mambular model
        with the default Mambular configuration.
        """,
        examples="""
        >>> from mambular.models import MambularClassifier
        >>> model = MambularClassifier(d_model=64, n_layers=8)
        >>> model.fit(X_train, y_train)
        >>> preds = model.predict(X_test)
        >>> model.evaluate(X_test, y_test)
        """,
    )

    def __init__(self, **kwargs):
        super().__init__(model=Mambular, config=DefaultMambularConfig, **kwargs)

In [24]:
classifier = MambularClassifier()

In [25]:
classifier.fit(
    X_train, y_train_enc,
    max_epochs=50,         
    batch_size=32,        
    patience=3,             
    monitor="val_loss",
    mode="min",
    lr=5e-4,               
    lr_patience=3,       
    lr_factor=0.5,
    weight_decay=1e-5,
);

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Numerical Feature: raw_h_sv2_mean, Info: {'preprocessing': 'imputer -> minmax -> ple', 'dimension': 20, 'categories': None}
--------------------------------------------------
Numerical Feature: raw_h_mag_mean, Info: {'preprocessing': 'imputer -> minmax -> ple', 'dimension': 20, 'categories': None}
--------------------------------------------------
Numerical Feature: raw_csi_matrix_r0_c0_mag_p95, Info: {'preprocessing': 'imputer -> minmax -> ple', 'dimension': 20, 'categories': None}
--------------------------------------------------
Numerical Feature: raw_csi_matrix_r0_c3_mag_p95, Info: {'preprocessing': 'imputer -> minmax -> ple', 'dimension': 20, 'categories': None}
--------------------------------------------------
Numerical Feature: raw_csi_matrix_r1_c3_mag_std, Info: {'preprocessing': 'imputer -> minmax -> ple', 'dimension': 20, 'categories': None}
--------------------------------------------------
Numerical Feature: raw_csi_matrix_r1_c1_spec_flatness, Info: {'preprocessing': 'imp

/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:751: Checkpoint directory /root/model_checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                      | Type             | Params | Mode 
-----------------------------------------------------------------------
0 | loss_fct                  | CrossEntropyLoss | 0      | train
1 | estimator                 | Mambular         | 340 K  | train
2 | estimator.embedding_layer | EmbeddingLayer   | 37.1 K | train
3 | estimator.mamba           | Mamba            | 302 K  | train
4 | estimator.tabular_head    | MLPhead          | 715    | train
-----------------------------------------------------------------------
340 K     Trainable params
0         Non-trainable params
340 K     Total params
1.361     Total estimated model params size (MB)
107       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.
/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


AssertionError: 

In [25]:
print(classifier.evaluate(X_test, y_test_enc))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

{'Accuracy': 0.8632793323514973}


In [32]:
# eval_viz_toolkit.py
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    classification_report, confusion_matrix, roc_curve, auc,
    precision_recall_curve, average_precision_score, top_k_accuracy_score
)
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from sklearn.calibration import calibration_curve

import torch
import textwrap  #
import joblib
import math 

sns.set_theme(style="whitegrid", font="DejaVu Sans", rc={
    "figure.dpi": 140,
    "savefig.dpi": 300,
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "legend.fontsize": 9,
})

def _ensure_dir(d):
    os.makedirs(d, exist_ok=True)

def _maybe_to_numpy(x):
    if isinstance(x, (pd.Series, pd.DataFrame)):
        return x.values
    return np.asarray(x)

def plot_training_curves_from_csvlog(csv_log_dir, save_dir):
    try:
        version_dirs = [d for d in os.listdir(csv_log_dir) if d.startswith("version_")]
        if not version_dirs:
            return
        latest = sorted(version_dirs, key=lambda x: int(x.split("_")[-1]))[-1]
        metrics_csv = os.path.join(csv_log_dir, latest, "metrics.csv")
        if not os.path.exists(metrics_csv):
            return
        df = pd.read_csv(metrics_csv)
      
        if "epoch" not in df.columns:
            return
        piv = df.pivot_table(index="epoch", columns="metric", values="value", aggfunc="mean")
        _ensure_dir(save_dir)

    
        plt.figure(figsize=(6,4))
        for col in piv.columns:
            if "loss" in str(col):
                plt.plot(piv.index, piv[col], label=col)
        plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Training/Validation Loss")
        plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(save_dir, "curve_loss.png")); plt.close()

      
        acc_like = [c for c in piv.columns if ("acc" in str(c).lower() or "accuracy" in str(c).lower())]
        if acc_like:
            plt.figure(figsize=(6,4))
            for col in acc_like:
                plt.plot(piv.index, piv[col], label=col)
            plt.xlabel("Epoch"); plt.ylabel("Score"); plt.title("Accuracy-like Curves")
            plt.legend(); plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "curve_acc.png")); plt.close()
    except Exception as e:
        print("[warn] plot_training_curves_from_csvlog failed:", e)

def _wrap_labels(labels, max_width=16):
    return [textwrap.fill(str(x), width=max_width, break_long_words=False) for x in labels]

def classification_report_figures(
    y_true,
    y_pred,
    class_names,
    save_path,                
    page_rows=15,          
    col_width=2.2,             
    row_height=0.7,            
    font_size=11,             
    title_font_size=16,       
    cmap="YlGnBu",             
    class_name_wrap=16,      
    save_pdf=True              
):

    # 1) 生成 report DataFrame
    rep = classification_report(
        y_true, y_pred,
        target_names=[str(c) for c in class_names],
        output_dict=True, zero_division=0
    )
    df = pd.DataFrame(rep).T


    tail_keys = ["accuracy", "macro avg", "weighted avg"]
    tail_idx = [i for i, idx in enumerate(df.index) if str(idx) in tail_keys]
    if tail_idx:
        split_pos = min(tail_idx)
        df_class = df.iloc[:split_pos, :].copy()
        df_overall = df.iloc[split_pos:, :].copy()
    else:
        df_class = df.copy()
        df_overall = pd.DataFrame(columns=df.columns)


    wrapped_names = _wrap_labels(df_class.index.tolist(), max_width=class_name_wrap)
    df_class.index = wrapped_names


    n_classes = df_class.shape[0]
    n_pages = math.ceil(n_classes / page_rows) if n_classes > 0 else 0
    base, ext = os.path.splitext(save_path)
    out_paths = []

    for p in range(n_pages):
        sl = slice(p * page_rows, min((p + 1) * page_rows, n_classes))
        page_df = df_class.iloc[sl, :].round(3)

        rows = page_df.shape[0]
        cols = page_df.shape[1]
        fig_w = max(8.0, cols * col_width + 2.0)
        fig_h = max(6.0, rows * row_height + 2.0)

        plt.figure(figsize=(fig_w, fig_h))
        sns.heatmap(
            page_df,
            annot=True, fmt=".3f",
            cmap=cmap, cbar=False,
            annot_kws={"size": font_size},
            linewidths=0.6, linecolor="#DDDDDD"
        )
        plt.title(f"Classification Report (Per-Class) — Page {p+1}/{n_pages}",
                  fontsize=title_font_size, pad=14)
        plt.xlabel("Metrics", fontsize=font_size+1)
        plt.ylabel("Classes", fontsize=font_size+1)
        plt.xticks(fontsize=font_size)
        plt.yticks(fontsize=font_size)

        plt.tight_layout()
        png_path = f"{base}_perclass_page{p+1}.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight")
        if save_pdf:
            pdf_path = f"{base}_perclass_page{p+1}.pdf"
            plt.savefig(pdf_path, dpi=300, bbox_inches="tight")
            out_paths.append(pdf_path)
        out_paths.append(png_path)
        plt.close()

    if not df_overall.empty:
        page_df = df_overall.round(3)
        rows = page_df.shape[0]
        cols = page_df.shape[1]
        fig_w = max(6.5, cols * (col_width*0.9) + 1.5)
        fig_h = max(3.8, rows * (row_height*1.1) + 1.2)

        plt.figure(figsize=(fig_w, fig_h))
        sns.heatmap(
            page_df,
            annot=True, fmt=".3f",
            cmap="Oranges", cbar=False,
            annot_kws={"size": font_size+1},
            linewidths=0.6, linecolor="#DDDDDD"
        )
        plt.title("Classification Report — Overall Summary", fontsize=title_font_size, pad=10)
        plt.xlabel("Metrics", fontsize=font_size+1)
        plt.ylabel("Summary", fontsize=font_size+1)
        plt.xticks(fontsize=font_size)
        plt.yticks(fontsize=font_size)

        plt.tight_layout()
        png_path = f"{base}_overall.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight")
        if save_pdf:
            pdf_path = f"{base}_overall.pdf"
            plt.savefig(pdf_path, dpi=300, bbox_inches="tight")
            out_paths.append(pdf_path)
        out_paths.append(png_path)
        plt.close()

    return out_paths

def plot_confusions(y_true, y_pred, class_names, save_dir):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))
    cmn = cm / cm.sum(axis=1, keepdims=True).clip(min=1)
    plt.figure(figsize=(6.5,5.5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix (Counts)")
    plt.tight_layout(); plt.savefig(os.path.join(save_dir, "cm_counts.png")); plt.close()

    plt.figure(figsize=(6.5,5.5))
    sns.heatmap(cmn, annot=True, fmt=".2f", cmap="Purples", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix (Normalized)")
    plt.tight_layout(); plt.savefig(os.path.join(save_dir, "cm_normalized.png")); plt.close()

def plot_roc_pr_curves(y_true, y_proba, class_names, save_dir):
    n_classes = len(class_names)
    y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))
    _ensure_dir(save_dir)

    # --- ROC ---
    fpr = dict(); tpr = dict(); roc_auc = dict()
    for i in range(n_classes):
        if y_true_bin[:, i].sum() == 0: 
            continue
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # micro
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_proba.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # macro
    fpr_grid = np.linspace(0, 1, 1000)
    tprs = []
    for k in fpr:
        if k in ("micro", "macro"):
            continue
        tprs.append(np.interp(fpr_grid, fpr[k], tpr[k]))
    tprs = np.array(tprs)
    tpr_mean = tprs.mean(axis=0) if len(tprs)>0 else np.zeros_like(fpr_grid)
    roc_auc["macro"] = auc(fpr_grid, tpr_mean)

    plt.figure(figsize=(7,5))
    plt.plot(fpr["micro"], tpr["micro"], label=f"micro-avg ROC (AUC={roc_auc['micro']:.3f})", lw=2)
    plt.plot(fpr_grid, tpr_mean, label=f"macro-avg ROC (AUC={roc_auc['macro']:.3f})", lw=2)
    for i in range(n_classes):
        if i in roc_auc:
            plt.plot(fpr[i], tpr[i], lw=1, alpha=0.6, label=f"{class_names[i]} (AUC={roc_auc[i]:.2f})")
    plt.plot([0,1],[0,1],"k--", lw=1)
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC Curves")
    plt.legend(ncol=2, fontsize=8); plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "roc_multi.png")); plt.close()

    # --- PR ---
    plt.figure(figsize=(7,5))
    ap_micro = average_precision_score(y_true_bin, y_proba, average="micro")
    plt.plot([0,1],[ap_micro, ap_micro], "k--", lw=1, label=f"micro-avg AP={ap_micro:.3f}")
    for i in range(n_classes):
        if y_true_bin[:, i].sum() == 0:
            continue
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_proba[:, i])
        ap = average_precision_score(y_true_bin[:, i], y_proba[:, i])
        plt.plot(recall, precision, lw=1.5, label=f"{class_names[i]} (AP={ap:.2f})")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall Curves")
    plt.legend(ncol=2, fontsize=8); plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "pr_multi.png")); plt.close()

def plot_topk_and_calibration(y_true, y_proba, save_dir, ks=(1,2,3,5)):
    _ensure_dir(save_dir)
    n_classes = y_proba.shape[1]
    # Top-K
    vals = []
    for k in ks:
        k = min(k, n_classes)
        vals.append(top_k_accuracy_score(y_true, y_proba, k=k, labels=np.arange(n_classes)))
    plt.figure(figsize=(5.5,4))
    plt.plot(list(ks), vals, marker="o")
    plt.xlabel("k"); plt.ylabel("Top-k Accuracy"); plt.title("Top-k Accuracy Curve")
    plt.grid(True, alpha=0.3); plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "topk.png")); plt.close()

    y_pred_conf = y_proba.max(axis=1)
    y_pred_cls  = y_proba.argmax(axis=1)
    correct     = (y_pred_cls == y_true).astype(int)
    prob_true, prob_pred = calibration_curve(correct, y_pred_conf, n_bins=15, strategy="quantile")
    plt.figure(figsize=(5.5,4))
    plt.plot([0,1],[0,1],"k--", alpha=0.5)
    plt.plot(prob_pred, prob_true, marker="o")
    plt.xlabel("Predicted confidence"); plt.ylabel("Empirical accuracy")
    plt.title("Reliability Diagram"); plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "calibration.png")); plt.close()

def plot_support_bars(y_true, class_names, save_path):
    cnts = pd.Series(y_true).value_counts().sort_index()
    plt.figure(figsize=(6,3.6))
    sns.barplot(x=class_names, y=cnts.values, color=sns.color_palette("Blues")[2])
    plt.xticks(rotation=45, ha="right"); plt.ylabel("Count"); plt.title("Class Support (Test)")
    plt.tight_layout(); plt.savefig(save_path); plt.close()

def plot_pca_embedding(preprocessor, X, y_true, y_pred, class_names, save_dir):
    """把经过预处理后的特征做 PCA 到2D，分别按 True / Pred 着色"""
    _ensure_dir(save_dir)
    try:
        X_t = preprocessor.transform(X)  
        X_t = np.asarray(X_t)
        n = min(8000, X_t.shape[0])       
        idx = np.random.choice(X_t.shape[0], n, replace=False) if X_t.shape[0] > n else np.arange(X_t.shape[0])
        X_sub = X_t[idx]; y_t = np.asarray(y_true)[idx]; y_p = np.asarray(y_pred)[idx]
        pca = PCA(n_components=2, random_state=42)
        Z = pca.fit_transform(X_sub)

        plt.figure(figsize=(6.2,5))
        sc = plt.scatter(Z[:,0], Z[:,1], c=y_t, s=10, cmap="tab20", alpha=0.75)
        plt.colorbar(sc, ticks=range(len(class_names)))
        plt.title("PCA (colored by True)"); plt.tight_layout()
        plt.savefig(os.path.join(save_dir, "pca_true.png")); plt.close()

        plt.figure(figsize=(6.2,5))
        sc = plt.scatter(Z[:,0], Z[:,1], c=y_p, s=10, cmap="tab20", alpha=0.75)
        plt.colorbar(sc, ticks=range(len(class_names)))
        plt.title("PCA (colored by Pred)"); plt.tight_layout()
        plt.savefig(os.path.join(save_dir, "pca_pred.png")); plt.close()
    except Exception as e:
        print("[warn] PCA embedding failed:", e)

def save_artifacts(classifier, class_names, save_dir):
    _ensure_dir(save_dir)
    try:
        net = classifier.task_model.estimator
        torch.save(net.state_dict(), os.path.join(save_dir, "model_weights.pt"))
    except Exception as e:
        print("[warn] saving model state_dict failed:", e)

    bundle = {
        "config": getattr(classifier, "config", None),
        "preprocessor": getattr(classifier, "preprocessor", None),
        "class_names": class_names,
    }
    joblib.dump(bundle, os.path.join(save_dir, "bundle.joblib"))

    try:
        with open(os.path.join(save_dir, "training_params.json"), "w", encoding="utf-8") as f:
            json.dump(classifier.get_params(), f, ensure_ascii=False, indent=2, default=str)
    except Exception as e:
        print("[warn] saving training_params.json failed:", e)

def full_evaluate_and_visualize(classifier, X_train, y_train_enc, X_test, y_test_enc, class_names, out_dir="artifacts"):
    _ensure_dir(out_dir)
    y_pred = classifier.predict(X_test)
    try:
        y_proba = classifier.predict_proba(X_test)
    except Exception:
        n_classes = len(class_names)
        y_proba = np.eye(n_classes)[y_pred]

    report_txt = classification_report(y_test_enc, y_pred, target_names=class_names, zero_division=0)
    with open(os.path.join(out_dir, "classification_report.txt"), "w", encoding="utf-8") as f:
        f.write(report_txt)

    plot_confusions(y_test_enc, y_pred, class_names, out_dir)
    classification_report_figures(y_test_enc, y_pred, class_names, os.path.join(out_dir, "classification_report.png"))
    plot_roc_pr_curves(y_test_enc, y_proba, class_names, out_dir)
    plot_topk_and_calibration(y_test_enc, y_proba, out_dir)
    plot_support_bars(y_test_enc, class_names, os.path.join(out_dir, "class_support.png"))
    plot_pca_embedding(classifier.preprocessor, X_test, y_test_enc, y_pred, class_names, out_dir)

    csv_log_dir = os.path.join(out_dir, "csv_logs")
    if os.path.isdir(csv_log_dir):
        plot_training_curves_from_csvlog(csv_log_dir, out_dir)

    save_artifacts(classifier, class_names, out_dir)

    print(f"[OK] 全部图与文件已输出到：{os.path.abspath(out_dir)}")


In [33]:


class_names = [str(c) for c in np.unique(y_train_enc)]
full_evaluate_and_visualize(
    classifier,
    X_train, y_train_enc,
    X_test,  y_test_enc,
    class_names=class_names,
    out_dir="q2_mamba"     
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

[warn] PCA embedding failed: tuple index out of range
[warn] saving training_params.json failed: 'Preprocessor' object has no attribute 'get_params'
[OK] 全部图与文件已输出到：/root/q2_mamba
